In [ ]:
# ---------------------------------------------------------------------------------------------
# I took over the Python CPU implementation for the bicubic interpolation algorithm
# from sanghavipreet - https://auth.geeksforgeeks.org/user/sanghavipreet2001/articles
# and I have made some changes (generalized the padding function for a given padding_number,
# renamed variables, removed/added comments).
# This file is under the same license as the original content from geeksforgeeks article.
# (see https://www.geeksforgeeks.org/copyright-information/?ref=footer) 
# (https://www.geeksforgeeks.org/xyz/)
# ---------------------------------------------------------------------------------------------

In [ ]:
import cv2
import numpy as np
import math
import sys
import time
input_image = cv2.imread('input_image.jpeg')
scale_factor = 2
# Interpolation kernel coefficient
a = -1/2

In [ ]:
# Interpolation kernel
def u(x,a):
    if (abs(x) >=0) & (abs(x) <=1):
        return (a+2)*(abs(x)**3)-(a+3)*(abs(x)**2)+1
    elif (abs(x) > 1) & (abs(x) <= 2):
        return a*(abs(x)**3)-(5*a)*(abs(x)**2)+(8*a)*abs(x)-4*a
    return 0

In [ ]:
# Paddig
def padding(img,H,W,C, padding_number):
    padded_img = np.zeros((H+padding_number*2,W+padding_number*2,C))
    padded_img[padding_number:H+padding_number,padding_number:W+padding_number,:C] = img
    padded_img[padding_number:H+padding_number,0:padding_number,:C]=img[:,0:1,:C]                         # the leftmost two col
    padded_img[H+padding_number:H+2*padding_number,padding_number:W+padding_number,:]=img[H-1:H,:,:]      # the top two rows
    padded_img[padding_number:H+padding_number,W+padding_number:W+2*padding_number,:]=img[:,W-1:W,:]      # the rightmost two col     
    padded_img[0:padding_number,padding_number:W+padding_number,:C]=img[0:1,:,:C]                         # the bottom two rows                              
    padded_img[0:padding_number,0:padding_number,:C]=img[0,0,:C]                                          # down left corner (4 points)               
    padded_img[H+padding_number:H+2*padding_number,0:padding_number,:C]=img[H-1,0,:C]                     # up left corner   (4 points)                    
    padded_img[H+padding_number:H+2*padding_number,W+padding_number:W+2*padding_number,:C]=img[H-1,W-1,:C]# up right corner  (4 points)                
    padded_img[0:padding_number,W+padding_number:W+2*padding_number,:C]=img[0,W-1,:C]                     # down right corner(4 points)  
    return padded_img

In [ ]:
# Bicubic algorithm
def bicubic(img, ratio, a):
    #tuple with array dimensions
    H,W,C = img.shape
    padding_number = 2
    img = padding(img,H,W,C, padding_number)
    #Create new image
    out_H = int(H*ratio) #rounds down to the nearest integer
    out_W = int(W*ratio)
    output_image = np.zeros((out_H, out_W, C))
    h = 1/ratio
    for c in range(C):
        for j in range(out_H):
            for i in range(out_W):
                x = i * h + 2
                y = j * h + 2
                x1 = 1 + x - math.floor(x)
                x2 = x - math.floor(x)
                x3 = math.floor(x) + 1 - x
                x4 = math.floor(x) + 2 - x
                y1 = 1 + y - math.floor(y)
                y2 = y - math.floor(y)
                y3 = math.floor(y) + 1 - y
                y4 = math.floor(y) + 2 - y
                mat_x = np.array([[u(x1,a),u(x2,a),u(x3,a),u(x4,a)]])
                # matrix with the nearest 16 points of the new pixel
                mat_pix = np.array([[img[int(y-y1),int(x-x1),c],img[int(y-y2),int(x-x1),c],img[int(y+y3),int(x-x1),c],img[int(y+y4),int(x-x1),c]],
                                   [img[int(y-y1),int(x-x2),c],img[int(y-y2),int(x-x2),c],img[int(y+y3),int(x-x2),c],img[int(y+y4),int(x-x2),c]],
                                   [img[int(y-y1),int(x+x3),c],img[int(y-y2),int(x+x3),c],img[int(y+y3),int(x+x3),c],img[int(y+y4),int(x+x3),c]],
                                   [img[int(y-y1),int(x+x4),c],img[int(y-y2),int(x+x4),c],img[int(y+y3),int(x+x4),c],img[int(y+y4),int(x+x4),c]]])
                mat_y = np.array([[u(y1,a)],[u(y2,a)],[u(y3,a)],[u(y4,a)]])
                output_image[j, i, c] = np.dot(np.dot(mat_x, mat_pix),mat_y)
    sys.stderr.write('\n')
    sys.stderr.flush()
    return output_image

In [ ]:
start_time = time.time()
output_image = bicubic(input_image, scale_factor, a)
stop_time = time.time()
sw_exec_time = stop_time - start_time
print('Bicubic CPU Python execution time: ',sw_exec_time)
#cv2.imshow('output_image',dst)
#cv2.waitKey(0)
#print('Completed!')
cv2.imwrite('output_image.jpeg', output_image)